In [ ]:
%load_ext autoreload
%autoreload 2

%cd '..'

In [ ]:
import logging
import sys

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    handlers=[
        logging.FileHandler(f"data/logs/overall_polarization.log"),
        logging.StreamHandler(stream=sys.stdout),
    ],
)


In [ ]:
import pandas as pd
from tqdm import tqdm

tqdm.pandas()

from polarization.utils import (
    calculate_polarization,
    calculate_polarization_by_time,
)
from preprocessing.utils import (
    build_vocab,
    save_event_vocab,
    load_event_vocab,
    tokenize_comment,
)
from preprocessing.constants import (
    OUTPUT_DIR,
    MIN_OCCURENCE_FOR_VOCAB,
)

from load.utils import (
    load_df_from_parquet,
    save_df_as_parquet,
)


In [ ]:
YEARS = [2013] #, 2013]  # 2014, 2015, 2016, 2017, 2018, 2019]

pol_by_month = []

for year in YEARS:
    logging.info(year)
    logging.info("Load partisan comments")  
    partisan_comments_year = load_df_from_parquet(
        file_name=f"user_partisan_comments_{year}.parquet",
    )

    partisan_comments_year = partisan_comments_year[
        partisan_comments_year["party"].isin(["dem", "rep"])
    ].copy()

    logging.info(f"Length of comments {year}: {len(partisan_comments_year)}")

    # logging.info("Building vocab...")
    # year_vocab = build_vocab(
    #     partisan_comments_year["tokens"],
    #     ngram_range=(1, 2),
    #     min_df=MIN_OCCURENCE_FOR_VOCAB,
    # )

    # logging.info("Saving vocab...")

    # # save vocab
    # save_event_vocab(
    #     year_vocab,
    #     theme="overall",
    #     event_name=f"user_partisan_comments_{year}",
    # )

    year_vocab = load_event_vocab(
        theme="overall",
        event_name=f"user_partisan_comments_{year}",
    )
    logging.info(f"Length of vocab {year}: {len(year_vocab)}")

    logging.info("Calculating leave-out polarization by month...")

    partisan_comments_year["datetime"] = pd.to_datetime(
        partisan_comments_year["created_utc"], unit="s"
    )

    logging.info(partisan_comments_year.dtypes)

    pol_by_month_year = calculate_polarization_by_time(
        partisan_comments_year,
        ngram_range=(1, 2),
        event_vocab=year_vocab,
        freq="M",
        equalize_users=True,
    )

    logging.info(f"Saving monthly polarization data {year}...")
    pol_by_month_year.to_parquet(
        f"{OUTPUT_DIR}/overall_{year}_leaveout_polarization_by_month.parquet",
        index=False,
    )

    pol_by_month.append(pol_by_month_year)

logging.info("Calculating leave-out polarization by month...")

pol_by_month = pd.concat(pol_by_month)

logging.info(f"Saving monthly polarization data...")

pol_by_month.to_parquet(
    f"{OUTPUT_DIR}/overall_leaveout_polarization_by_month.parquet",
    index=False,
)

## Plot

In [ ]:
YEARS = [2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019]

logging.info("Load partisan comments")

pol_by_month = []

for year in YEARS:
    logging.info(year)
    logging.info("Load partisan comments")  
    pol_by_month_year = load_df_from_parquet(
        file_name=f"overall_{year}_leaveout_polarization_by_month.parquet",
    )

    pol_by_month_year["date"] = pd.to_datetime(pol_by_month_year["date"])

    pol_by_month.append(pol_by_month_year)


pol_years_by_month = pd.concat(pol_by_month)


In [ ]:
display(pol_years_by_month)


In [ ]:
# Line plot

import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme()

plt.figure(figsize=(12, 5))


ax = sns.lineplot(
    data=pol_years_by_month,
    y="polarization",
    x="date",
)

sns.lineplot(
    data=pol_years_by_month,
    y="random_polarization",
    x="date",
)

plt.ylabel("Leave-out partisanship estimate")
plt.xlabel("Date")

plt.savefig(
    "data/figures/polarization/overall/overall_monthly_partisanship.pdf",
    bbox_inches='tight'
)

plt.show()
